In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [77]:
# Load trained model
model = load_model('model.hS', compile=False)

# Load geography one-hot encoder
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

# Load gender label encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

# Load scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [78]:
input_data = pd.DataFrame([{
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',   # MUST be string
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumofProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}])

print(input_data)
print(input_data.dtypes)

   CreditScore Geography Gender  Age  Tenure  Balance  NumofProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  
CreditScore         int64
Geography          object
Gender             object
Age                 int64
Tenure              int64
Balance             int64
NumofProducts       int64
HasCrCard           int64
IsActiveMember      int64
EstimatedSalary     int64
dtype: object


In [79]:
print(label_encoder_gender.classes_)

['Female' 'Male']


In [80]:
input_data['Gender'] = label_encoder_gender.transform(
    input_data['Gender']
)

print(input_data)

   CreditScore Geography  Gender  Age  Tenure  Balance  NumofProducts  \
0          600    France       1   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000  


In [81]:



# Encode Geography (OneHotEncoder)
geo_encoded = onehot_encoder_geo.transform(input_data[['Geography']])

geo_encoded_df = pd.DataFrame(
    geo_encoded.toarray(),
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [82]:
input_df = pd.DataFrame([input_data])
input_df

ValueError: Must pass 2-d input. shape=(1, 1, 10)

In [ ]:
input_data=pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumofProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
input_data['Gender'] = label_encoder_gender.transform(
    input_data['Gender']
)
input_data

ValueError: y contains previously unseen labels: 1

In [ ]:
input_data = pd.concat(
    [input_data.drop("Geography", axis=1), geo_encoded_df],
    axis=1
)
input_data

KeyError: "['Geography'] not found in axis"

In [ ]:
input_scaled=scaler.transform(input_data)
input_scaled

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- NumofProducts
Feature names seen at fit time, yet now missing:
- NumOfProducts


In [ ]:
input_data = input_data.rename(columns={
    'NumofProducts': 'NumOfProducts'
})

In [83]:
input_data = pd.DataFrame([{
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,   # ✅ CORRECT NAME
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}])

In [ ]:
# encode gender
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])

# one-hot geography
geo_encoded = onehot_encoder_geo.transform(
    input_data[['Geography']]
).toarray()

geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

# drop geography & concat
input_data = pd.concat(
    [input_data.drop('Geography', axis=1), geo_encoded_df],
    axis=1
)

# 🔥 CRITICAL: reorder columns
input_data = input_data[scaler.feature_names_in_]

# scale
input_scaled = scaler.transform(input_data)

In [84]:
print(set(input_data.columns) - set(scaler.feature_names_in_))
print(set(scaler.feature_names_in_) - set(input_data.columns))

{'Geography'}
{'Geography_Germany', 'Geography_Spain', 'Geography_France'}


In [85]:
# 🔥 CRITICAL: reorder columns
input_data = input_data[scaler.feature_names_in_]

# scale
input_scaled = scaler.transform(input_data)

KeyError: "['Geography_France', 'Geography_Germany', 'Geography_Spain'] not in index"